# sociodemographic




# Librerias

In [1]:
# Datos
import numpy as np # Calculos matriciales
import pandas as pd # Manejo de dataframes


# Visualizacion
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from random import random
import folium




# Preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.feature_selection import VarianceThreshold
from scipy import stats


# Evaluacion
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics


import warnings
warnings.filterwarnings("ignore")

# Load dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_sd = pd.read_csv('/content/drive/MyDrive/Nuclio/Capstone/data_easymoney/data/sociodemographic_df.csv')
df_sd.head()

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01-28,ES,29.0,H,35,N,87218.10
1,1,1050611,2018-01-28,ES,13.0,V,23,N,35548.74
2,2,1050612,2018-01-28,ES,13.0,V,23,N,122179.11
3,3,1050613,2018-01-28,ES,50.0,H,22,N,119775.54
4,4,1050614,2018-01-28,ES,50.0,V,23,N,NaN


In [4]:
df_sd.shape

(5962924, 9)

# ML Preprossing

## Explorar Datos

info(), describe(), head()

In [5]:
# Obtener información sobre el conjunto de datos
df_sd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    int64  
 1   pk_cid        int64  
 2   pk_partition  object 
 3   country_id    object 
 4   region_code   float64
 5   gender        object 
 6   age           int64  
 7   deceased      object 
 8   salary        float64
dtypes: float64(2), int64(3), object(4)
memory usage: 409.4+ MB


In [6]:
# Estadísticas descriptivas
df_sd.describe

<bound method NDFrame.describe of          Unnamed: 0   pk_cid pk_partition country_id  region_code gender  age  \
0                 0  1375586   2018-01-28         ES         29.0      H   35   
1                 1  1050611   2018-01-28         ES         13.0      V   23   
2                 2  1050612   2018-01-28         ES         13.0      V   23   
3                 3  1050613   2018-01-28         ES         50.0      H   22   
4                 4  1050614   2018-01-28         ES         50.0      V   23   
...             ...      ...          ...        ...          ...    ...  ...   
5962919    13647304  1166765   2019-05-28         ES         50.0      V   22   
5962920    13647305  1166764   2019-05-28         ES         26.0      V   23   
5962921    13647306  1166763   2019-05-28         ES         50.0      H   47   
5962922    13647307  1166789   2019-05-28         ES         50.0      H   22   
5962923    13647308  1550586   2019-05-28         ES         28.0      H   37   

        deceased     salary  
0              N   87218.10  
1              N   35548.74  
2              N  122179.11  
3              N  119775.54  
4              N        NaN  
...          ...        ...  
5962919        N   43912.17  
5962920        N   23334.99  
5962921        N        NaN  
5962922        N  199592.82  
5962923        N        NaN  

[5962924 rows x 9 columns]>

In [7]:
# Explorar los primeros registros
df_sd.head()

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01-28,ES,29.0,H,35,N,87218.10
1,1,1050611,2018-01-28,ES,13.0,V,23,N,35548.74
2,2,1050612,2018-01-28,ES,13.0,V,23,N,122179.11
3,3,1050613,2018-01-28,ES,50.0,H,22,N,119775.54
4,4,1050614,2018-01-28,ES,50.0,V,23,N,NaN



```
0 - Unnamed: 0,int64,
1 - pk_cid - int64 - Identificador de cliente
2 - pk_partition - object - Fecha de ingesta de los datos
3 - country_id - object - País de residencia del cliente
4 - region_code - float64 - Provincia de residencia del cliente (para ES)
5 - gender - object - Sexo del cliente
6 - age - int64 - Edad del cliente
7 - deceased - object - Deceased index. N/S
8 - salary - float64 - Ingresos brutos de la unidad familiar
```


## Data Cleaning

El step del data cleaning es el cajón de sastre donde corregiremos las particularidades del dataset más allá del ML preprocessing estandar. Por ejemplo:

Eliminar registros duplicados
Eliminar filas sin target informado
Eliminar columnas irrelevantes
Corregir erratas en literales
Corregir formatos de fecha
Corregir data types

### Eliminar registros duplicados

In [8]:
df_sd2 = df_sd.copy()

In [9]:
# Check duplicados
len(df_sd2.index.unique())==len(df_sd2.index)

True

In [10]:
df_sd2.duplicated().sum()

0

## EDA

Analisis de las distribuciones de las variables.
* Imputar valores nulos
* Corrección de formatos
* Gestión de altas correlaciones
* Bajas varianzas

In [12]:
df_sd3 = df_sd2.copy()

### Imputar Nulos

Algunas de las estrategias mas comunes:

* Imputar valor outlier (-999, 'Unknown')
* Imputar media en numericos
* Imputar moda en categoricos
* Imputar 0

In [13]:
# Sumamos la cantidad de nulos
df_sd3.isna().sum()

Unnamed: 0            0
pk_cid                0
pk_partition          0
country_id            0
region_code        2264
gender               25
age                   0
deceased              0
salary          1512103
dtype: int64

In [14]:
# Variables correslacionadas
corr = df_sd3.corr(numeric_only=True)

In [15]:
corr

,Unnamed: 0,pk_cid,region_code,age,salary
Unnamed: 0,1.000000,0.191033,0.001787,0.018059,0.005747
pk_cid,0.191033,1.000000,0.041581,-0.117927,-0.017251
region_code,0.001787,0.041581,1.000000,0.018516,-0.018625
age,0.018059,-0.117927,0.018516,1.000000,0.014022
salary,0.005747,-0.017251,-0.018625,0.014022,1.000000


#### "region_code

In [16]:
df_sd3.head()

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01-28,ES,29.0,H,35,N,87218.10
1,1,1050611,2018-01-28,ES,13.0,V,23,N,35548.74
2,2,1050612,2018-01-28,ES,13.0,V,23,N,122179.11
3,3,1050613,2018-01-28,ES,50.0,H,22,N,119775.54
4,4,1050614,2018-01-28,ES,50.0,V,23,N,NaN


In [17]:
# Sumamos la cantidad de nulos en 'region_code'
df_sd3['region_code'].isna().sum()

2264

In [18]:
# Contar la cantidad de 'cp' por cada código de país
count_by_country = df_sd3.groupby('country_id')['region_code'].nunique()

# Imprimir el resultado
print(count_by_country)

country_id
AR     0
AT     0
BE     0
BR     0
CA     0
CH     0
CI     0
CL     0
CM     0
CN     0
CO     0
DE     0
DJ     0
DO     0
DZ     0
ES    52
ET     0
FR     0
GA     0
GB     0
GT     0
HU     0
IE     0
IT     0
JM     0
LU     0
MA     0
MR     0
MX     0
NO     0
PE     0
PL     0
PT     0
QA     0
RO     0
RU     0
SA     0
SE     0
SN     0
US     0
VE     0
Name: region_code, dtype: int64


In [19]:
# Contamos los valores unicos que country_id
df_sd3['country_id'].value_counts()

ES    5960672
GB        441
FR        225
DE        199
US        195
CH        194
BR         87
BE         81
VE         79
IE         68
MX         58
AT         51
AR         51
PL         49
IT         45
MA         34
CL         30
CN         28
CA         22
LU         17
ET         17
QA         17
CI         17
SA         17
CM         17
SN         17
MR         17
NO         17
RU         17
CO         17
GA         17
GT         17
DO         17
SE         16
DJ         11
PT         11
JM         11
RO          9
HU          8
DZ          7
PE          4
Name: country_id, dtype: int64

In [20]:
modas_por_grupo = df_sd3.groupby('country_id')['region_code'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None)

In [21]:
modas_por_grupo

country_id
AR     NaN
AT     NaN
BE     NaN
BR     NaN
CA     NaN
CH     NaN
CI     NaN
CL     NaN
CM     NaN
CN     NaN
CO     NaN
DE     NaN
DJ     NaN
DO     NaN
DZ     NaN
ES    28.0
ET     NaN
FR     NaN
GA     NaN
GB     NaN
GT     NaN
HU     NaN
IE     NaN
IT     NaN
JM     NaN
LU     NaN
MA     NaN
MR     NaN
MX     NaN
NO     NaN
PE     NaN
PL     NaN
PT     NaN
QA     NaN
RO     NaN
RU     NaN
SA     NaN
SE     NaN
SN     NaN
US     NaN
VE     NaN
Name: region_code, dtype: float64

In [22]:
# Rellenar nulos en 'region_code' con la moda según 'country_id'
df_sd3['region_code'] = df_sd3.apply(lambda row: modas_por_grupo[row['country_id']] if pd.isnull(row['region_code']) else row['region_code'], axis=1)

In [23]:
# Sumamos la cantidad de nulos en 'gender'
df_sd3['region_code'].isna().sum()

2252

In [24]:
#A los null pendientes les imputamos un Valores Outlier 999
Values_999_region_code = {
    "region_code": 999,
}

In [25]:
#Asiganmos los nuevos valores a los N/A
df_sd3 = df_sd3.fillna(Values_999_region_code)

In [26]:
# Otra opcion es eliminar la fila
#df_sd3.dropna(subset=['region_code'], inplace=True)

In [27]:
# Validamos la cantidad de nulos
df_sd3.isna().sum()

Unnamed: 0            0
pk_cid                0
pk_partition          0
country_id            0
region_code           0
gender               25
age                   0
deceased              0
salary          1512103
dtype: int64

#### "gender"

In [28]:
df_sd3.head()

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01-28,ES,29.0,H,35,N,87218.10
1,1,1050611,2018-01-28,ES,13.0,V,23,N,35548.74
2,2,1050612,2018-01-28,ES,13.0,V,23,N,122179.11
3,3,1050613,2018-01-28,ES,50.0,H,22,N,119775.54
4,4,1050614,2018-01-28,ES,50.0,V,23,N,NaN


In [29]:
# Contamos los valores unicos que hay
df_sd3['gender'].unique()

array(['H', 'V', nan], dtype=object)

In [30]:
# Sumamos la cantidad de nulos en 'gender'
df_sd3['gender'].isna().sum()

25

In [31]:
# Analisis de "gender"
df_sd3["gender"].value_counts()

H    3087502
V    2875397
Name: gender, dtype: int64

In [32]:
# Calcular la moda de columna 'gender' (puede haber múltiples modas)
mode_gender = df_sd3['gender'].mode()

In [33]:
# Obtener el valor de la moda (usaremos el primer valor si hay múltiples modas)
mode_gender = mode_gender[0] if not mode_gender.empty else None

In [34]:
mode_gender

'H'

In [35]:
# Imputamos la moda de gender en los valores nulos
df_sd3['gender'] = df_sd3['gender'].fillna(mode_gender)

In [36]:
# Validamos la cantidad de nulos
df_sd3.isna().sum()

Unnamed: 0            0
pk_cid                0
pk_partition          0
country_id            0
region_code           0
gender                0
age                   0
deceased              0
salary          1512103
dtype: int64

#### "salary"

In [37]:
df_sd3.head()

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01-28,ES,29.0,H,35,N,87218.10
1,1,1050611,2018-01-28,ES,13.0,V,23,N,35548.74
2,2,1050612,2018-01-28,ES,13.0,V,23,N,122179.11
3,3,1050613,2018-01-28,ES,50.0,H,22,N,119775.54
4,4,1050614,2018-01-28,ES,50.0,V,23,N,NaN


In [38]:
media_columna = df_sd3['salary'].mean()

In [39]:
media_columna

115816.72370447844

In [44]:
# Saco la media de 'salary' segun 'region_code'
mean_por_grupo = df_sd3.groupby('region_code')['salary'].median()

In [45]:
mean_por_grupo

region_code
1.0       74453.79
2.0       76811.73
3.0       63001.98
4.0       69529.59
5.0       67608.57
6.0       60310.89
7.0      116585.25
8.0      124630.71
9.0       87680.88
10.0      66398.85
11.0      75568.29
12.0      64863.84
13.0      60880.05
14.0      64438.98
15.0      94066.14
16.0      67490.40
17.0     108188.76
18.0      78106.23
19.0      91690.26
20.0      53028.75
21.0      67356.51
22.0      72616.44
23.0      64164.72
24.0      73127.46
25.0      64152.54
26.0      85056.24
27.0      64600.71
28.0     126762.48
29.0      89968.53
30.0      65932.26
31.0      77641.98
32.0      78325.26
33.0      83582.13
34.0      84224.01
35.0      77092.32
36.0      96886.86
37.0      86346.96
38.0      76277.61
39.0      89202.00
40.0      89031.21
41.0      86136.03
42.0      79147.80
43.0      83784.96
44.0      76702.80
45.0      65672.31
46.0      68052.93
47.0      88935.39
48.0     108659.43
49.0      73414.65
50.0      96796.23
51.0     124158.42
52.0     114363.99


In [46]:
# Imputa los valores nulos usando la media por grupo
df_sd3['salary'] = df_sd3['salary'].fillna(df_sd3['region_code'].map(mean_por_grupo))

In [47]:
# Validamos la cantidad de nulos
df_sd3.isna().sum()

Unnamed: 0      0
pk_cid          0
pk_partition    0
country_id      0
region_code     0
gender          0
age             0
deceased        0
salary          0
dtype: int64

### Corrección de formatos

Encoding categorico
3 opciones:

* Si tienen un sentido ordinal -> convertir a numerico.
* Si tienen un sentido cardinal:
** 2 posibles categorias -> indicador booleano
** más de 2 categorias -> One-Hot-Encoding

In [101]:
df_sd4 = df_sd3.copy()

In [102]:
df_sd4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Unnamed: 0    int64  
 1   pk_cid        int64  
 2   pk_partition  object 
 3   country_id    object 
 4   region_code   float64
 5   gender        object 
 6   age           int64  
 7   deceased      object 
 8   salary        float64
dtypes: float64(2), int64(3), object(4)
memory usage: 409.4+ MB


In [103]:
df_sd4.shape

(5962924, 9)

In [104]:
df_sd4.head()

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,0,1375586,2018-01-28,ES,29.0,H,35,N,87218.10
1,1,1050611,2018-01-28,ES,13.0,V,23,N,35548.74
2,2,1050612,2018-01-28,ES,13.0,V,23,N,122179.11
3,3,1050613,2018-01-28,ES,50.0,H,22,N,119775.54
4,4,1050614,2018-01-28,ES,50.0,V,23,N,96796.23


In [105]:
def obtener_lista_variables(df_sd4):
  lista_num = []
  lista_bool = []
  lista_cat = []

  for i in df_sd4:
    if (df_sd4[i].dtype.kind in "i" or df_sd4[i].dtype.kind=="f") and \
    len(df_sd4[i].unique()) != 2:
      lista_num.append(i)
    elif (df_sd4[i].dtype.kind=="i" or df_sd4[i].dtype.kind=="f") and \
    len(df_sd4[i].unique()) == 2:
      lista_bool.append(i)
    elif (df_sd4[i].dtype.kind=="O" ):
      lista_cat.append(i)

  return lista_num, lista_bool, lista_cat

In [106]:
lnum, lbool, lcat = obtener_lista_variables(df_sd4)

In [107]:
lnum

['Unnamed: 0', 'pk_cid', 'region_code', 'age', 'salary']

In [108]:
lbool

[]

In [109]:
lcat

['pk_partition', 'country_id', 'gender', 'deceased']


```
0 - Unnamed: 0,int64,
1 - pk_cid - int64 - Identificador de cliente
2 - pk_partition - object - Fecha de ingesta de los datos
3 - country_id - object - País de residencia del cliente
4 - region_code - float64 - Provincia de residencia del cliente (para ES)
5 - gender - object - Sexo del cliente
6 - age - int64 - Edad del cliente
7 - deceased - object - Deceased index. N/S
8 - salary - float64 - Ingresos brutos de la unidad familiar
```

gender

In [110]:
# Contamos los valores unicos que hay
df_sd4['gender'].unique()

array(['H', 'V'], dtype=object)

In [111]:
# Definimos crear una nueva columna 'bool_gender' y asignar los valores {'H': 0, 'V': 1}
df_sd4['bool_gender'] = df_sd4['gender'].map({'H': 0, 'V': 1})

In [112]:
df_sd4 = df_sd4.drop('gender', axis=1)

In [113]:
df_sd4.head()

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,age,deceased,salary,bool_gender
0,0,1375586,2018-01-28,ES,29.0,35,N,87218.10,0
1,1,1050611,2018-01-28,ES,13.0,23,N,35548.74,1
2,2,1050612,2018-01-28,ES,13.0,23,N,122179.11,1
3,3,1050613,2018-01-28,ES,50.0,22,N,119775.54,0
4,4,1050614,2018-01-28,ES,50.0,23,N,96796.23,1


deceased

In [114]:
# Contamos los valores unicos que hay
df_sd4['deceased'].unique()

array(['N', 'S'], dtype=object)

In [115]:
# Definimos crear una nueva columna 'bool_deceased' y asignar los valores {'N': 0, 'S': 1}
df_sd4['bool_deceased'] = df_sd4['deceased'].map({'N': 0, 'S': 1})

In [116]:
df_sd4 = df_sd4.drop('deceased', axis=1)

In [117]:
df_sd4.head()

,Unnamed: 0,pk_cid,pk_partition,country_id,region_code,age,salary,bool_gender,bool_deceased
0,0,1375586,2018-01-28,ES,29.0,35,87218.10,0,0
1,1,1050611,2018-01-28,ES,13.0,23,35548.74,1,0
2,2,1050612,2018-01-28,ES,13.0,23,122179.11,1,0
3,3,1050613,2018-01-28,ES,50.0,22,119775.54,0,0
4,4,1050614,2018-01-28,ES,50.0,23,96796.23,1,0


country_id

In [118]:
# Contamos los valores unicos que hay
df_sd4['country_id'].unique()

array(['ES', 'CA', 'CH', 'CL', 'IE', 'AT', 'FR', 'GB', 'DE', 'DO', 'BE',
       'AR', 'VE', 'US', 'MX', 'PL', 'MA', 'GT', 'GA', 'CO', 'BR', 'RU',
       'IT', 'NO', 'SN', 'MR', 'ET', 'CN', 'CM', 'SA', 'CI', 'QA', 'LU',
       'SE', 'DJ', 'PT', 'JM', 'RO', 'HU', 'DZ', 'PE'], dtype=object)

In [121]:
len(df_sd4['country_id'].unique())

41

In [122]:
# Definir la tabla de mapeo
values_to_impute_country_id = {
    "ES": 0,
    "CA": 1,
    "CH": 2,
    "CL": 3,
    "IE": 4,
    "AT": 5,
    "FR": 6,
    "GB": 7,
    "DE": 8,
    "DO": 9,
    "BE": 10,
    "AR": 11,
    "VE": 12,
    "US": 13,
    "MX": 14,
    "PL": 15,
    "MA": 16,
    "GT": 17,
    "GA": 18,
    "CO": 19,
    "BR": 20,
    "RU": 21,
    "IT": 22,
    "NO": 23,
    "SN": 24,
    "MR": 25,
    "ET": 26,
    "CN": 27,
    "CM": 28,
    "SA": 29,
    "CI": 30,
    "QA": 31,
    "LU": 32,
    "SE": 33,
    "DJ": 34,
    "PT": 35,
    "JM": 36,
    "RO": 37,
    "HU": 38,
    "DZ": 39,
    "PE": 40
}

In [123]:
# Aplicar el mapeo a la columna 'country_id'
df_sd4['country_id_num'] = df_sd4['country_id'].map(values_to_impute_country_id)

In [127]:
df_sd4 = df_sd4.drop('country_id', axis=1)

In [128]:
df_sd4.head()

,Unnamed: 0,pk_cid,pk_partition,region_code,age,salary,bool_gender,bool_deceased,country_id_num
0,0,1375586,2018-01-28,29.0,35,87218.10,0,0,0
1,1,1050611,2018-01-28,13.0,23,35548.74,1,0,0
2,2,1050612,2018-01-28,13.0,23,122179.11,1,0,0
3,3,1050613,2018-01-28,50.0,22,119775.54,0,0,0
4,4,1050614,2018-01-28,50.0,23,96796.23,1,0,0


pk_partition

In [131]:
# Convertir la columna 'pk_partition' al formato de fecha
df_sd4['pk_partition'] = pd.to_datetime(df_sd4['pk_partition'])

In [132]:
df_sd4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5962924 entries, 0 to 5962923
Data columns (total 9 columns):
 #   Column          Dtype         
---  ------          -----         
 0   Unnamed: 0      int64         
 1   pk_cid          int64         
 2   pk_partition    datetime64[ns]
 3   region_code     float64       
 4   age             int64         
 5   salary          float64       
 6   bool_gender     int64         
 7   bool_deceased   int64         
 8   country_id_num  int64         
dtypes: datetime64[ns](1), float64(2), int64(6)
memory usage: 409.4 MB


In [133]:
# Crear las nuevas columnas
df_sd4['pk_partition_dia'] = df_sd4['pk_partition'].dt.day
df_sd4['pk_partition_mes'] = df_sd4['pk_partition'].dt.month
df_sd4['pk_partition_año'] = df_sd4['pk_partition'].dt.year

In [135]:
df_sd4 = df_sd4.drop('pk_partition', axis=1)

In [136]:
df_sd4.head()

,Unnamed: 0,pk_cid,region_code,age,salary,bool_gender,bool_deceased,country_id_num,pk_partition_dia,pk_partition_mes,pk_partition_año
0,0,1375586,29.0,35,87218.10,0,0,0,28,1,2018
1,1,1050611,13.0,23,35548.74,1,0,0,28,1,2018
2,2,1050612,13.0,23,122179.11,1,0,0,28,1,2018
3,3,1050613,50.0,22,119775.54,0,0,0,28,1,2018
4,4,1050614,50.0,23,96796.23,1,0,0,28,1,2018


Unnamed: 0

In [137]:
n_index = 'Index'
df_sd4 = df_sd4.rename_axis(n_index).reset_index()

In [138]:
df_sd4.set_index('Index', inplace=True)
del(df_sd4["Unnamed: 0"])

In [139]:
df_sd4.head()

,pk_cid,region_code,age,salary,bool_gender,bool_deceased,country_id_num,pk_partition_dia,pk_partition_mes,pk_partition_año
Index,,,,,,,,,,
0,1375586,29.0,35,87218.10,0,0,0,28,1,2018
1,1050611,13.0,23,35548.74,1,0,0,28,1,2018
2,1050612,13.0,23,122179.11,1,0,0,28,1,2018
3,1050613,50.0,22,119775.54,0,0,0,28,1,2018
4,1050614,50.0,23,96796.23,1,0,0,28,1,2018
